In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
print(plt.style.available)
%matplotlib inline
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'  # 设置加载策略为 eager
driver = webdriver.Chrome(options=chrome_options)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


There was an error managing chromedriver (error decoding response body); using driver found in the cache


In [3]:
url_esf='https://esf.fang.com/'
driver.get(url_esf)

In [4]:
Haidian_esf=driver.find_element(By.XPATH,'/html/body/div[4]/div[3]/div[2]/div[1]/ul/li[1]/ul/li[2]/a')
Haidian_esf.click()

In [5]:
Shangdi_esf=driver.find_element(By.XPATH,'/html/body/div[4]/div[3]/div[2]/div[1]/ul/li[2]/ul/li[17]/a')
driver.execute_script("arguments[0].click();", Shangdi_esf)

In [6]:
#mine data on esf website in 20 pages
i = 0
Num_Pages = 20
house_esf_data=[]
while i < Num_Pages:
    location='上地'
    house_items=driver.find_elements(By.CSS_SELECTOR, "div.shop_list dl.clearfix")
    for item in house_items:
        try:
            area_element = item.find_element('class name','tel_shop')
            area_text = area_element.text
            import re
            area_match = re.search(r'(\d+\.?\d*)\s*㎡', area_text)
            area = area_match.group(1) if area_match else None
            
            total_price_element = item.find_element(By.CSS_SELECTOR, "dd.price_right span.red b")
            total_price = total_price_element.text if total_price_element else None

            price_element = item.find_element(By.CSS_SELECTOR, "dd.price_right span:not(.red)")
            unit_price = price_element.text if price_element else None
            price_match1 = re.search(r'(\d+\.?\d*)\s*元/㎡', unit_price)
            unit_price = price_match1.group(1) if price_match1 else None
            
            house_esf_data.append({
                    'location': location,
                    'square': area,
                    'total':total_price,
                    'unit': unit_price

                })
        except Exception as e:
                print(f"fail extracting information: {e}")
                continue
    try:
        next_page = driver.find_element(By.LINK_TEXT, '下一页')
        driver.execute_script("arguments[0].click();", next_page)
        i += 1
    except NoSuchElementException:
        break

df_esf = pd.DataFrame(house_esf_data)

In [7]:
df_esf.head()

,location,square,total,unit
0,上地,125.3,1099,87709
1,上地,149.73,1950,130234
2,上地,108.82,698,64142
3,上地,146.25,1068,73025
4,上地,191,1260,65968


In [8]:
df_esf

,location,square,total,unit
0,上地,125.3,1099,87709
1,上地,149.73,1950,130234
2,上地,108.82,698,64142
3,上地,146.25,1068,73025
4,上地,191,1260,65968
...,...,...,...,...
1195,上地,132.13,1000,75684
1196,上地,84.33,710,84193
1197,上地,86.59,599,69177
1198,上地,44.31,382,86211


In [9]:
df_esf.describe()

,location,square,total,unit
count,1200,1200,1200,1200
unique,1,285,200,451
top,上地,111.57,760,110101
freq,1200,31,28,12


In [10]:
df_esf[['square','unit']].describe()

,square,unit
count,1200,1200
unique,285,451
top,111.57,110101
freq,31,12


In [11]:
#open renting house website
url_rent='https://zu.fang.com'
driver.get(url_rent)

In [12]:
Haidian_rent=driver.find_element(By.XPATH,'/html/body/div[4]/div[2]/div[2]/div/dl[1]/dd/a[3]')
driver.execute_script("arguments[0].click();", Haidian_rent)

In [13]:
Shangdi_rent=driver.find_element(By.XPATH,'//*[@id="rentid_D04_08"]/a[20]')
driver.execute_script("arguments[0].click();", Shangdi_rent)

In [14]:
# mine data on rent website in 20 pages
j = 0
Num_Pages = 20
house_rent_data=[]
while j < Num_Pages:
    location='上地'
    house_items=driver.find_elements(By.CSS_SELECTOR, "div.houseList dl.list.hiddenMap.rel")
    for item in house_items:
        try:
            area_element = item.find_element(By.CSS_SELECTOR,'p.font15.mt12.bold')
            area_text = area_element.text
            import re
            area_match = re.search(r'(\d+\.?\d*)\s*㎡', area_text)
            area = area_match.group(1) if area_match else None
        
            price_element = item.find_element('class name', "price")
            price_permonth = price_element.text if price_element else None

            house_rent_data.append({
                    'location': location,
                    'square': area,
                    'rent': price_permonth
                })
        except Exception as e:
                print(f"fail extracting information: {e}")
                continue
    try:
        next_page = driver.find_element(By.LINK_TEXT, '下一页')
        driver.execute_script("arguments[0].click();", next_page)
        j += 1
    except NoSuchElementException:
        break

df_rent = pd.DataFrame(house_rent_data)

In [15]:
df_rent.head()

,location,square,rent
0,上地,314,25500
1,上地,114,10500
2,上地,136,12900
3,上地,106,12000
4,上地,125,11000


In [16]:
df_rent.describe()

,location,square,rent
count,957,957,957
unique,1,134,171
top,上地,59,12000
freq,957,35,54


In [17]:
df_esf.to_excel('df_esf_Shangdi.xlsx', index=False)

In [18]:
df_rent.to_excel('df_rent_Shangdi.xlsx', index=False)